# Customer Outreach Campaign

---

## Setup

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_type = os.environ.get("OPENAI_API_TYPE")
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = os.environ.get("OPENAI_API_VERSION")

## LLM

In [ ]:
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    deployment_name="gpt40125",
    temperature=0,
)

## Creating Agents

In [4]:
from crewai import Agent, Task, Crew

In [5]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    llm=llm,
    allow_delegation=False,
    verbose=True
)

In [6]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    llm=llm,
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [7]:
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool

In [8]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [9]:
from crewai_tools import BaseTool


class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [10]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

In [11]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [12]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [ ]:
crew = Crew(
  agents=[sales_rep_agent, lead_sales_rep_agent],
  tasks=[lead_profiling_task, personalized_outreach_task],
  verbose=2,
	memory=True,
  embedder={
      "provider": "azure_openai",
      "config": {
          "model": "text-embedding-3-small",
          "deployment_name": "text-embedding-3-small",
      }
  }
)

## Running the Crew

In [14]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
To start, I need to gather initial information about DeepLearningAI, focusing on their background, key personnel, recent milestones, and potential needs. This will help in understanding how our solutions can add value to their operations. The first step is to search the internet for general information about DeepLearningAI.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI compan

In [15]:
from IPython.display import Markdown

Markdown(result)

---

**Email 1: Subject: Empowering AI Education Together**

Dear Andrew,

I hope this message finds you well. I've been closely following DeepLearningAI's remarkable journey in transforming AI education, making it accessible to learners worldwide. Your recent collaborations to expand your educational offerings are truly inspiring.

At CrewAI, we share your passion for democratizing AI education. Our AI-driven educational tools are designed to offer immersive and interactive learning experiences, which I believe could complement DeepLearningAI's curriculum beautifully. Our platforms could provide your students with hands-on training opportunities, making complex AI concepts more tangible.

Would you be open to a brief discussion on how we can collaborate to further enrich the AI learning experience? I'm looking forward to exploring the possibilities together.

Best regards,

[Your Name]  
Lead Sales Representative, CrewAI

---

**Email 2: Subject: Enhancing AI Learning with Advanced Analytics**

Dear Andrew,

Following our previous conversation about integrating CrewAI's solutions into DeepLearningAI's courses, I wanted to share how our AI analytics could add another layer of value. Our tools can offer deep insights into student engagement and performance, enabling you to tailor your courses for maximum effectiveness.

Imagine having the ability to adjust your curriculum in real-time based on comprehensive analytics, ensuring that every learner's needs are met. This could be a game-changer in achieving our shared goal of advancing AI education.

I'd love to discuss this further and perhaps arrange a demo to showcase what our AI analytics can do. Let me know if this interests you.

Warm regards,

[Your Name]  
Lead Sales Representative, CrewAI

---

**Email 3: Subject: A Pilot Project Proposal for DeepLearningAI**

Dear Andrew,

I've been thinking about our discussions on how CrewAI's solutions can support DeepLearningAI's mission. What if we started with a pilot project? This would allow us to integrate our AI tools into one of your courses as a test case, demonstrating the practical benefits and the enhanced learning experience they can provide.

This pilot could serve as a tangible example of how our collaboration could evolve, with minimal risk and high potential for impactful results. We're fully committed to supporting you in this venture, providing all necessary resources to ensure its success.

Would you be interested in exploring this idea further? I believe it could pave the way for a fruitful partnership between CrewAI and DeepLearningAI.

Looking forward to your thoughts,

[Your Name]  
Lead Sales Representative, CrewAI

---

These emails are crafted to gradually build interest and engagement, starting from an introduction of shared values and moving towards more concrete proposals for collaboration. Each message is personalized to resonate with Andrew Ng's known priorities and DeepLearningAI's mission, ensuring the tone is engaging, professional, and aligned with their corporate identity.